In [ ]:
from shapely.geometry import box, Point, MultiPoint, Polygon
from typing import List
from tqdm import tqdm

import plotly.graph_objects as go
import geopandas as gpd
import pandas as pd
import numpy as np

import os
import pdal
import shapely
import json
import laspy
import random

In [ ]:
%cd ..
%cd assets
assert os.getcwd().split("/")[-1] == 'assets', "You are not in the assets directory"

In [ ]:
def visualize_3d_array(point_cloud_array: np.ndarray=None, file_name_list: List=None, example_ID=None):
    
    x = point_cloud_array[example_ID, :, 0].flatten()
    y = point_cloud_array[example_ID, :, 1].flatten()
    z = point_cloud_array[example_ID, :, 2].flatten()

    scatter = go.Scatter3d(x=x, y=y, z=z, mode='markers', 
                         marker=dict(size = 3, color = z, colorscale = 'Viridis'))
    layout = go.Layout(title = f'Visualization of {file_name_list[example_ID]}')
    fig = go.Figure(data = [scatter], layout = layout)
    fig.show()

def _convert_numpy_to_las(x: np.ndarray=None, header=None):
    
    outfile = laspy.LasData(header)
    outfile.x = x[:,0]
    outfile.y = x[:,1]
    outfile.z = x[:,2]
    outfile.intensity = x[:,3]
    outfile.raw_classification = x[:,4]
    outfile.scan_angle_rank = x[:,5]
    
    return outfile

def _sample_random_points(x: np.ndarray=None, random_sample_size: int=None):
    
    rng = np.random.default_rng()
    lidar_subset = rng.choice(a=x, size=random_sample_size, replace=False, axis=0)
    
    return lidar_subset

def _convert_las_to_numpy(las_data = None):
    
    lidar_numpy = np.array((las_data.x, 
                             las_data.y, 
                             las_data.z, 
                             las_data.intensity, 
                             las_data.raw_classification, 
                             las_data.scan_angle_rank)).transpose()
    
    return lidar_numpy
        
def subsample_las(original_las_data_filepath: str=None, random_sample_size: int=1000000):

    org_las_data = laspy.read(original_las_data_filepath)
    # Set meta data for new LAS file based on settings from original LAS file
    hdr = org_las_data.header
    hdr.point_count = 0
    
    lidar_ndarray = _convert_las_to_numpy(las_data = org_las_data)
    print(f"SHAPE of LIDAR: {lidar_ndarray.shape}")
    
    lidar_subset = _sample_random_points(x=lidar_ndarray, random_sample_size=random_sample_size)
    print(f"SHAPE of LIDAR_SUBSET: {lidar_subset.shape}")
    
    outfile = _convert_numpy_to_las(lidar_subset, hdr)
    output_filepath = original_las_data_filepath[:-4] + "_SUBSET.las"
    
    print(f"Saving subsampled LAS file to: {output_filepath}")
    outfile.write(output_filepath)
    
    return outfile

def create_tile_bounding_box(original_las_data_filepath: str=None):
    
    las_data = laspy.read(original_las_data_filepath)
    min_x, min_y, min_z, max_x, max_y, max_z = [*las_data.header.min, *las_data.header.max]
    return box(minx=min_x, miny=min_y, maxx=max_x, maxy=max_y)

In [ ]:
def transform_ndarray_to_gpd_dataframe(numpy_point_cloud: np.ndarray=None, target_crs: int=None):
    
    if numpy_point_cloud.ndim == 3:
        numpy_point_cloud = np.squeeze(numpy_point_cloud)
        
    pandas_point_cloud = pd.DataFrame(numpy_point_cloud)
    
    geometry_3D = [shapely.geometry.Point(xyz) for xyz in zip(pandas_point_cloud[0], pandas_point_cloud[1], pandas_point_cloud[2])]
    # CRS in m for UK
    geopandas_point_cloud = gpd.GeoDataFrame(geometry_3D, crs=27700, geometry=geometry_3D) 
    geopandas_point_cloud = geopandas_point_cloud.drop(columns=[0])
    geopandas_point_cloud = geopandas_point_cloud.to_crs(target_crs)
    
    return geopandas_point_cloud

def transform_gpd_dataframe_to_multipoint(geopandas_point_cloud: gpd.GeoDataFrame=None):
    
    coord_list = [(point.x, point.y, point.z) for point in geopandas_point_cloud.geometry.tolist()]
    
    multipoint = MultiPoint(coord_list)
    
    return multipoint

def find_min_z_value(numpy_point_cloud: np.ndarray=None):
    
    z_values = numpy_point_cloud[:,2]
    min_z = np.min(z_values)

    return min_z

In [ ]:
# Load building footprints
osm_footprints = gpd.read_file("coventry_building_footprints.geojson")
osm_footprints

In [ ]:
# Load LiDAR point cloud
file_path = "/Users/kevin/Projects/CS224W_LIDAR/assets/cropped_16.las"
las_point_cloud = laspy.read(file_path)
numpy_point_cloud = _convert_las_to_numpy(las_point_cloud)
min_z = find_min_z_value(numpy_point_cloud)
numpy_point_cloud = numpy_point_cloud[np.newaxis, ...]
print(f"numpy_point_cloud shape: {numpy_point_cloud.shape}")
# Visualize building point cloud
visualize_3d_array(point_cloud_array=numpy_point_cloud, file_name_list=[file_path], example_ID=0)

In [ ]:
# Convert numpy point cloud to geopandas point cloud
geopandas_point_cloud = transform_ndarray_to_gpd_dataframe(numpy_point_cloud=numpy_point_cloud, target_crs=27700)

# Group building-level points into MultiPoint object
lidar_multipoint = transform_gpd_dataframe_to_multipoint(geopandas_point_cloud)

In [ ]:
# Find respective footprint for LiDAR point cloud
building_footprint = None

for elem in osm_footprints.geometry:
    
    if isinstance(elem, shapely.geometry.Polygon):
        
        if elem.contains(geopandas_point_cloud.loc[1000,'geometry']):
        
            building_footprint = elem
            
building_footprint

In [ ]:
# Grid spacing in meters
resolution = 0.5

lonmin, latmin, lonmax, latmax = building_footprint.bounds

# construct rectangle of points
x, y = np.round(np.meshgrid(np.arange(lonmin, lonmax, resolution), np.arange(latmin, latmax, resolution)), 4)
points = list(zip(x.flatten(),y.flatten()))

# validate each point falls inside shapes and add minimum z-vale of lidar point cloud as floor level
valid_points = [(point[0], point[1], min_z) for point in points if building_footprint.contains(shapely.geometry.Point(point))]

In [ ]:
footprint_multipoint = shapely.geometry.MultiPoint(valid_points)

footprint_multipoint

In [ ]:
from shapely.ops import unary_union

new_multipoint = unary_union([lidar_multipoint, footprint_multipoint])

new_multipoint

In [ ]:
new_numpy_point_cloud = np.array([(point.x, point.y, point.z) for point in list(new_multipoint)])
new_numpy_point_cloud.shape

In [ ]:
new_numpy_point_cloud = new_numpy_point_cloud[np.newaxis, ...]
# Visualize building point cloud
visualize_3d_array(point_cloud_array=new_numpy_point_cloud, file_name_list=[file_path], example_ID=0)